In [63]:
from pathlib import Path

# ----- Read Excel file ----- #

REL_PATH = Path("InputData/CoreData.xlsx")

def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    """
    Starting at cwd, walk up to `max_up` parents to find `rel_path`.
    Returns the resolved path if found; raises FileNotFoundError otherwise.
    """
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        candidate = (parent / rel_path)
        if candidate.exists():
            return candidate.resolve()
    # Helpful diagnostics
    raise FileNotFoundError(
        f"Couldn't locate '{rel_path.as_posix()}' from {here} by walking up {max_up} levels.\n"
        f"- Current working directory: {here}\n"
        f"- Checked: {[str((p / rel_path)) for p in [here, *here.parents][: max_up + 1]]}"
    )

INPUT_XLSX = find_upwards(REL_PATH)

import pandas as pd
xfile = pd.ExcelFile(INPUT_XLSX)
sheets = xfile.sheet_names
print("Resolved path:", INPUT_XLSX)
print("Sheets:", sheets)

assert isinstance(sheets, list), "Expected a list"
assert sheets and all(isinstance(s, str) and s.strip() for s in sheets), "Sheet names must be non-empty strings"
assert len(sheets) == len(set(sheets)), "Duplicate sheet names detected"
print("Check 1 passed.")


Resolved path: /Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Studium TUM/Master Management and Technology/06 Master Thesis/00 Thesis/05Code/InputData/CoreData.xlsx
Sheets: ['Metadata', 'dashboard', 'general_partner', 'fund', 'fund_cash_flow', 'capital_account', 'deal', 'deal_time_series', 'deal_cash_flow', 'deal_partner', 'deal_acquirer', 'deal_vendor', 'organization', 'person']
Check 1 passed.


In [64]:
import pandas as pd

#----- Quickly check whether also the sheets get read correctly -----#

SHEET = "deal_time_series"

# Guard: make sure the sheet exists (uses `sheets` from Step 1)
assert SHEET in sheets, f"'{SHEET}' not found. Available sheets: {sheets}"

# Read with first row as header
dts = pd.read_excel(INPUT_XLSX, sheet_name=SHEET, header=0)

# Display column names and first 10 data rows (i.e., Excel rows 2–11)
print("Column names:", list(dts.columns))
display(dts.head(5))

# --- Check 2
assert isinstance(dts, pd.DataFrame), "Expected a pandas DataFrame."
assert not dts.empty, "Sheet loaded but contains no data."
assert all(isinstance(c, str) and c.strip() for c in dts.columns), "Invalid/empty column names."
print(f"Check 2 passed. Shape: {dts.shape}. Showing 5 data rows above.")


Column names: ['id', 'total_value', 'ebitda', 'reference_period_type_suffix', 'moic_gross', 'data_room_id', 'created_by_user_id', 'recurring_revenue', 'bridge_financing', 'reporting_currency_financials', 'irr_net', 'reference_period_type_prefix', 'moic_net', 'data_room_name', 'realized_value', 'irr_gross', 'ebitda_adjusted', 'net_debt', 'ebitda_multiple', 'enterprise_value_valuation_rationale', 'is_main', 'equity', 'reporting_currency_valuation', 'management_equity_percentage', 'revenue_multiple', 'recurring_revenue_percentage', 'quarterly_company_update', '_created_at_utc', 'enterprise_value', 'enterprise_value_valuation_multiple', '_year', 'reference_date', 'irr_net_unlevered', 'capex', 'total_investment_cost', 'deal_revision_id', 'reported_date', 'deal_id', 'unrealized_value', 'predicted_sentiment', 'ebit', 'ebitda_adjusted_note', 'ebitda_margin', 'cumulative_addons', 'moic_net_unlevered', 'revenue', 'fund_equity_invested', '_quarter', '_revision_id', 'ownership_economic_percentage'

,id,total_value,ebitda,reference_period_type_suffix,moic_gross,data_room_id,created_by_user_id,recurring_revenue,bridge_financing,reporting_currency_financials,...,ebitda_adjusted_note,ebitda_margin,cumulative_addons,moic_net_unlevered,revenue,fund_equity_invested,_quarter,_revision_id,ownership_economic_percentage,enterprise_value_valuation_amount
0,b0cd7032-72f6-46d0-ae21-7a0ca81297eb,NaN,NaN,Actual,NaN,203ffba5-3ebb-454a-844c-87cee656bd95,25bd1583-7869-465c-9dc4-664685cd3a6c,NaN,NaN,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,1,cccb3423-eb71-452a-92f1-3b4a64100646,NaN,NaN
1,997bb98e-9ab1-47be-b04b-767d225f60a9,NaN,7600000.0,Actual,NaN,d40592d4-9127-4e77-a8c7-9da4755a6105,25bd1583-7869-465c-9dc4-664685cd3a6c,NaN,NaN,EUR,...,NaN,0.046,NaN,NaN,164000000.0,NaN,1,64ef422a-f3cc-44fe-bf2b-fe5955950008,NaN,NaN
2,1eab0e13-6d1d-4c99-a8a0-6c32b56de012,NaN,34000000.0,Actual,NaN,d40592d4-9127-4e77-a8c7-9da4755a6105,25bd1583-7869-465c-9dc4-664685cd3a6c,NaN,NaN,EUR,...,NaN,0.121,NaN,NaN,280000000.0,NaN,2,3e0b9962-f308-420f-9c62-5fa24f5b2e7e,NaN,NaN
3,9ae2e65b-ec9d-4215-8982-f39658e9fa1e,19839000.0,13747260.0,NaN,NaN,9d92005d-7097-4949-88ea-8eb6ff688a86,38ed8bb8-d707-4652-94c0-6a094d116b50,NaN,NaN,EUR,...,NaN,NaN,NaN,NaN,48663900.0,19839000.0,3,9ae2e65b-ec9d-4215-8982-f39658e9fa1e,1.0,NaN
4,fa677592-875d-413e-91fe-c8af1dd99f63,40736000.0,18894000.0,NaN,NaN,9d92005d-7097-4949-88ea-8eb6ff688a86,38ed8bb8-d707-4652-94c0-6a094d116b50,NaN,NaN,EUR,...,NaN,NaN,NaN,NaN,393249800.0,40736000.0,2,fa677592-875d-413e-91fe-c8af1dd99f63,0.5,NaN


Check 2 passed. Shape: (4956, 51). Showing 5 data rows above.


In [65]:
from pathlib import Path
import pandas as pd

#----- Create output CSV and take the deal_time_series_id as starter column -----#

# Config
SHEET = "deal_time_series"
TARGET_DIR = (find_upwards(Path("ValueCreation")) / "Data")
TARGET_DIR.mkdir(parents=True, exist_ok=True)
TARGET_CSV = TARGET_DIR / "working.csv"

assert SHEET in sheets, f"Sheet '{SHEET}' not found."

# Load only the key column; no filtering, no sorting
usecols = ["id"]
raw = pd.read_excel(INPUT_XLSX, sheet_name=SHEET, usecols=usecols)

# Preserve order exactly as in the sheet
df = raw[["id"]]

# Persist
df.to_csv(TARGET_CSV, index=False)
print(f"Wrote {len(df):,} rows to {TARGET_CSV}")


import pandas as pd

assert TARGET_CSV.exists(), f"Missing output: {TARGET_CSV}"
check_df = pd.read_csv(TARGET_CSV)

# 1) Columns exactly as specified and in order
assert list(check_df.columns) == ["id"], list(check_df.columns)

# 2) Row count preserved
assert len(check_df) == len(raw), f"Row count changed: raw={len(raw)} vs written={len(check_df)}"

# 3) Order preserved: id sequence identical pre/post write
assert check_df["id"].tolist() == raw["id"].tolist(), "Row order changed."

# 4) Key integrity: non-null and unique
assert check_df["id"].notna().all(), "Null id found."
assert not check_df["id"].duplicated().any(), "Duplicate id values found."

print("INIT check passed. Shape:", check_df.shape)


Wrote 4,956 rows to /Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Studium TUM/Master Management and Technology/06 Master Thesis/00 Thesis/05Code/ValueCreation/Data/working.csv
INIT check passed. Shape: (4956, 1)


In [66]:
from pathlib import Path
import pandas as pd
import numpy as np

# ----- Add all needed columns from deal_time_series to the output CSV -----#

SHEET = "deal_time_series"
TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# Load working file and source
working = pd.read_csv(TARGET_CSV, dtype={"id": str})

requested = [
    "deal_id", "reference_date", "enterprise_value", "net_debt", "equity", "reporting_currency_financials", "reference_period_type_prefix", "reference_period_type_suffix", "revenue", "ebitda", "ownership_economic_percentage", "data_room_name",
]
src = pd.read_excel(
    INPUT_XLSX,
    sheet_name=SHEET,
    usecols=["id", *requested],
    dtype={"id": str},
)

# Parse Excel-serial 'reference_date' to ISO yyyy-mm-dd (CSV-friendly)
if "reference_date" in src.columns:
    s = src["reference_date"]
    if np.issubdtype(s.dtype, np.number):
        dt = pd.to_datetime(s, unit="D", origin="1899-12-30", errors="coerce")
    else:
        dt = pd.to_datetime(s, errors="coerce")
    src["reference_date"] = dt.dt.strftime("%Y-%m-%d")

# Only add columns not already present
to_add = [c for c in requested if c not in working.columns]
src = src[["id", *to_add]]

# Preserve original row order
working["_ord"] = np.arange(len(working))

# Left-join on id
out = working.merge(src, on="id", how="left")

# Restore order and drop helper
out = out.sort_values("_ord").drop(columns="_ord")

# Save
out.to_csv(TARGET_CSV, index=False)
print(f"Added columns: {to_add}. Wrote {len(out):,} rows to {TARGET_CSV}.")

after = pd.read_csv(TARGET_CSV, dtype={"id": str})

# Row count preserved
assert len(after) == len(working), "Row count changed."

# Order preserved
assert after["id"].tolist() == working.sort_values("_ord")["id"].tolist(), "Order changed."

# Requested columns present
missing = [c for c in requested if c not in after.columns]
assert not missing, f"Missing columns: {missing}"

# reference_date parses or is blank
_ = pd.to_datetime(after["reference_date"], errors="coerce")
print("ADD_COLUMNS (deal_time_series extra fields) check passed. Shape:", after.shape)


Added columns: ['deal_id', 'reference_date', 'enterprise_value', 'net_debt', 'equity', 'reporting_currency_financials', 'reference_period_type_prefix', 'reference_period_type_suffix', 'revenue', 'ebitda', 'ownership_economic_percentage', 'data_room_name']. Wrote 4,956 rows to /Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Studium TUM/Master Management and Technology/06 Master Thesis/00 Thesis/05Code/ValueCreation/Data/working.csv.
ADD_COLUMNS (deal_time_series extra fields) check passed. Shape: (4956, 13)


In [67]:
from pathlib import Path
import pandas as pd
import numpy as np

# ----- Add all needed columns from deal to the output CSV -----#

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")
working = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})

requested = [
    "fund_id", "name", "entry_date", "entry_transaction_type",
    "sourcing_type", "investment_role", "exit_date", "exit_transaction_type",
]
src = pd.read_excel(
    INPUT_XLSX,
    sheet_name="deal",
    usecols=["id", *requested],
    dtype={"id": str},
)

# Only add columns that aren't already present
to_add = [c for c in requested if c not in working.columns]
src = src[["id", *to_add]].rename(columns={"id": "deal_id"})

# Preserve original order
working["_ord"] = np.arange(len(working))

# Left-join on deal_id
out = working.merge(src, on="deal_id", how="left")

# Restore order and drop helper
out = out.sort_values("_ord").drop(columns="_ord")

# Save
out.to_csv(TARGET_CSV, index=False)
print(f"Added columns from 'deal': {to_add}. Wrote {len(out):,} rows to {TARGET_CSV}.")

after = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})

# Row count preserved
assert len(after) == len(working), "Row count changed."

# Order preserved
assert after["id"].tolist() == working.sort_values("_ord")["id"].tolist(), "Order changed."

# Requested columns present
missing = [c for c in to_add if c not in after.columns]
assert not missing, f"Missing columns after merge: {missing}"

print("ADD_COLUMNS (deal) check passed. Shape:", after.shape)



Added columns from 'deal': ['fund_id', 'name', 'entry_date', 'entry_transaction_type', 'sourcing_type', 'investment_role', 'exit_date', 'exit_transaction_type']. Wrote 4,956 rows to /Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Studium TUM/Master Management and Technology/06 Master Thesis/00 Thesis/05Code/ValueCreation/Data/working.csv.
ADD_COLUMNS (deal) check passed. Shape: (4956, 21)


In [68]:
from pathlib import Path
import pandas as pd
import numpy as np

# ----- Add all needed columns from fund to the output CSV -----#

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# Load working file
working = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str, "fund_id": str})

requested = ["name", "vintage_year", "investment_theme", "size", "fund_generation", "fund_family_generation"]
src = pd.read_excel(
    INPUT_XLSX,
    sheet_name="fund",
    usecols=["id", *requested],
    dtype={"id": str},
)

# Rename to avoid collision and to align join key
src = src.rename(columns={"id": "fund_id", "name": "fund_name"})

# Only add columns that aren't already present
to_add = [c if c != "name" else "fund_name" for c in requested]
to_add = [c for c in to_add if c not in working.columns]
src = src[["fund_id", *to_add]]

# Preserve original order
working["_ord"] = np.arange(len(working))

# Left join on fund_id
out = working.merge(src, on="fund_id", how="left")

# Restore order and drop helper
out = out.sort_values("_ord").drop(columns="_ord")

# Save
out.to_csv(TARGET_CSV, index=False)
print(f"Added columns from 'fund': {to_add}. Wrote {len(out):,} rows to {TARGET_CSV}.")

after = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str, "fund_id": str})

# Row count preserved
assert len(after) == len(working), "Row count changed."

# Order preserved
assert after["id"].tolist() == working.sort_values("_ord")["id"].tolist(), "Order changed."

# Requested columns (with rename) present
expected = [c if c != "name" else "fund_name" for c in ["name","vintage_year","investment_theme","size","fund_generation","fund_family_generation"]]
missing = [c for c in expected if c not in after.columns]
assert not missing, f"Missing columns after merge: {missing}"

print("ADD_COLUMNS (fund) check passed. Shape:", after.shape)


Added columns from 'fund': ['fund_name', 'vintage_year', 'investment_theme', 'size', 'fund_generation', 'fund_family_generation']. Wrote 4,956 rows to /Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Studium TUM/Master Management and Technology/06 Master Thesis/00 Thesis/05Code/ValueCreation/Data/working.csv.
ADD_COLUMNS (fund) check passed. Shape: (4956, 27)


In [69]:
from pathlib import Path
import pandas as pd

#----- Quick test to get all rows of one deal_id in the output CSV -----#

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

deal_id_value = "49328606-087c-4288-972a-614c19bd519e"
# "49328606-087c-4288-972a-614c19bd519e" #NA test DEAL
# "30f4104d-0343-4031-a729-ec81b646861a" Visma DEAL
df = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})
subset = df.loc[df["deal_id"] == deal_id_value]

print(f"Rows for deal_id={deal_id_value}: {len(subset)}")
display(subset)

assert not subset.empty, "No rows found for the specified deal_id."
print("Filter check passed.")

"""from pathlib import Path
import pandas as pd

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

name_value = "Visma DEAL"

df = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})
subset = df.loc[df["name"].astype(str).str.strip() == name_value]

print(f'Rows for name="{name_value}": {len(subset)}')
display(subset)

assert "name" in df.columns, "Column 'name' not found."
assert not subset.empty, f'No rows found for name="{name_value}".'
print("Filter check passed.")"""


Rows for deal_id=49328606-087c-4288-972a-614c19bd519e: 1


,id,deal_id,reference_date,enterprise_value,net_debt,equity,reporting_currency_financials,reference_period_type_prefix,reference_period_type_suffix,revenue,...,sourcing_type,investment_role,exit_date,exit_transaction_type,fund_name,vintage_year,investment_theme,size,fund_generation,fund_family_generation
4069,693b03b7-f6f0-4540-8b7c-07e1ac4fff65,49328606-087c-4288-972a-614c19bd519e,2024-03-01,NaN,NaN,NaN,EUR,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filter check passed.


'from pathlib import Path\nimport pandas as pd\n\nTARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")\n\nname_value = "Visma DEAL"\n\ndf = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})\nsubset = df.loc[df["name"].astype(str).str.strip() == name_value]\n\nprint(f\'Rows for name="{name_value}": {len(subset)}\')\ndisplay(subset)\n\nassert "name" in df.columns, "Column \'name\' not found."\nassert not subset.empty, f\'No rows found for name="{name_value}".\'\nprint("Filter check passed.")'